本例研究多线程、多进程下，多大量图片resize的性能比较

结果显示：使用多线程或多进程，可以提高resize对批量图片的处理速度

In [1]:
import concurrent.futures as cf
import random
import time

import cv2
import numpy as np

# 每张图片的处理过程
def resize_image(i,w,h,type=1):
    img_path=F'images/{i}.jpg'
    img=cv2.imread(img_path,1)
    
    img=cv2.resize(img,(w,h),interpolation=type)
    
    img=img.astype(np.float64)
    # 亮度变化
    delta = random.uniform(-32, 32)
    img += delta
    # 对比度变化
    alpha = random.uniform(0.5, 1.5)
    img *= alpha
    
    # 归一化及标准化
    img=img/255.0
    mean=(0.406, 0.456, 0.485)
    std=(0.225, 0.224, 0.229)
    img -= mean
    img /= std

In [2]:
# 传递参数
image_size=20
image_idx=[i%image_size+1 for i in range(image_size*image_size)]
resize_w=[512 for i in range(image_size*image_size)]
resize_h=[512 for i in range(image_size*image_size)]

In [3]:
start = time.time()
#without multiprocessing 普通调用
for i in range(image_size*image_size):
    resize_image(i%image_size+1,512,512)
print("{} seconds".format(round(time.time() - start,2)))

7.01 seconds


In [4]:
start = time.time()
#using I/O bottleneck 多线程
with cf.ThreadPoolExecutor(max_workers=2) as p:
    processing = p.map(resize_image,image_idx,resize_w,resize_h)
print("{} seconds".format(round(time.time() - start,2)))

3.85 seconds


In [5]:
start = time.time()
#using CPU bottleneck 多进程
with cf.ProcessPoolExecutor(max_workers=2) as p:
    processing = p.map(resize_image,image_idx,resize_w,resize_h)
print("{} seconds".format(round(time.time() - start,2)))

3.73 seconds
